In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot
%matplotlib inline

from collections import Counter

In [4]:
df = pd.read_csv("../Frequent_Itemsets/data/sample.txt", sep= "\t", encoding = "ISO-8859-1", header=None)
df.columns = ['revenue','title','timestamp','customer_id']

In [5]:
def preprocessing(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.dropna()
    return df

In [6]:
df_new = preprocessing(df)
print(df_new.shape, df.shape)
print("Unique number of customers:", df['customer_id'].nunique())
df_new.head()

(2969511, 4) (2988222, 4)
Unique number of customers: 1437925


,revenue,title,timestamp,customer_id
0,1.67,Sport ist Mord,2019-07-05,A2UAJ8NW9XIIS0
1,0.83,The First Avenger: Civil War [dt./OV],2019-07-14,AIDSKOPJNZJ93
3,3.35,Michael Clayton,2019-07-13,A290LGW9N2RY19
4,0.83,Deadpool [dt./OV],2019-07-12,A3VIQZ7YAKNZR0
5,0.83,Männertag,2019-07-13,A1Z9CMUOVNZL5


In [7]:
def get_combinations(df):
    
    df = df.sort_values(by = ['customer_id','timestamp'])
    
    t = pd.DataFrame(df.groupby("customer_id")['title'].agg(lambda x: x.tolist())).reset_index()
    t.columns = ['customer_id','combination']
    
    #Get intersection in case you have have 2 column of lists
    #df['intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df.A, df.B)]
    
    
    t['combination'] = t.apply(lambda row: set(row['combination']), axis=1)    
    
    count_combo = t['combination'].apply(tuple)
    d = Counter(elem for elem in count_combo)
    
    df_output = pd.DataFrame.from_dict(d, orient='index').reset_index()
    
    df_output.columns = ['combination','count']
    return df_output, t

In [8]:
a,b = get_combinations(df_new)

In [27]:
b.head()

,customer_id,combination
0,A0000366A0GK5998YBDT,{Die Royal Tenenbaums}
1,A0000434260Q6LEE180GF,{Warcraft: The Beginning [dt./OV]}
2,A0001210LXBR1BZ5BP3,"{Polaroid, Bailey - Ein Freund fürs Leben [dt...."
3,A00019461IMC9V3PF4AFN,{Bohemian Rhapsody [dt./OV]}
4,A00021686F07H1Z9ODO7,{Bent - Korruption kennt keine Regeln}


In [19]:
c = a.sort_values(by = "count", ascending = False)
c['count_elements'] = c.apply(lambda row: len(row['combination']), axis=1)

In [20]:
c.head()

,combination,count,count_elements
48,"(25 km/h,)",18764,1
195,(Drachenzähmen leicht gemacht 3: Die geheime W...,11121,1
189,"(Spider-Man: A New Universe [dt./OV],)",10906,1
88,"(Pets [dt./OV],)",10581,1
190,"(Mary Poppins? Rückkehr [dt./OV],)",9229,1


In [25]:
c[c['count_elements'] == 2][0:20]

,combination,count,count_elements
139,"(John Wick [dt./OV], John Wick: Kapitel 2 [dt....",1910,2
610,(Die Tribute von Panem - Mockingjay Teil 1 [dt...,1288,2
364,(Drachenzähmen leicht gemacht 3: Die geheime W...,939,2
2803,"(Paw Patrol - Staffel 4 Teil 2 [dt./OV], Paw P...",883,2
57,"(Träume von Schäfchen, Was sagen die Gene?)",713,2
450,"(Mary Poppins? Rückkehr [dt./OV], Spider-Man: ...",697,2
247,"(Game of Thrones - Staffel 8 [dt./OV], Game of...",666,2
8077,"(25 km/h, Mary Poppins? Rückkehr [dt./OV])",649,2
3303,"(25 km/h, Sauerkrautkoma)",645,2
2021,"(Die Hüterin der Wahrheit ? Dinas Bestimmung, ...",605,2


In [36]:
c.loc[57]['combination']

('Träume von Schäfchen', 'Was sagen die Gene?')